# Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [1]:
import pandas as pd
from statadict import parse_stata_dict
import time
import numpy as np
import pickle
from collections import defaultdict

## Examples from Chapter 1

Read NSFG data into a Pandas DataFrame.

In [2]:
def GetDataframe():
    try:
        with open('preg_df.pickle', 'rb') as read_file:
            preg_df = pickle.load(read_file)
    except:

        stata_dict = parse_stata_dict(file="2002FemPreg.dct")
        data = pd.read_fwf("2002FemPreg.dat", names=stata_dict.names, colspecs=stata_dict.colspecs)
        data['prglength'] = (data.apply(lambda x:x['wksgest'] if x['wksgest'] else x['mosgest']*4.33, axis=1))

        preg_df = data[['caseid', 'prglength', 'outcome', 'pregordr',
                                        'birthord', 'birthwgt_lb', 'birthwgt_oz',
                                        'agepreg', 'finalwgt']]

        with open('preg_df.pickle', 'wb') as file:
            pickle.dump(obj=preg_df, file=file)
            
    return preg_df


In [3]:
# read the dataframe
preg_df = GetDataframe()
preg_df.head()

,caseid,prglength,outcome,pregordr,birthord,birthwgt_lb,birthwgt_oz,agepreg,finalwgt
0,1,39.0,1,1,1.0,8.0,13.0,3316.0,6448.271112
1,1,39.0,1,2,2.0,7.0,14.0,3925.0,6448.271112
2,2,39.0,1,1,1.0,9.0,2.0,1433.0,12999.542264
3,2,39.0,1,2,2.0,7.0,0.0,1783.0,12999.542264
4,2,39.0,1,3,3.0,6.0,3.0,1833.0,12999.542264


In [9]:
def data_cleaning():
    preg_df['agepreg'] = preg_df['agepreg'].apply(lambda x: x / 100)
    na_vals = [97, 98, 99]
    preg_df.loc[preg_df['birthwgt_lb'] > 20, 'birthwgt_lb'] = np.nan
    preg_df['birthwgt_lb'] = preg_df['birthwgt_lb'].apply(lambda x:x if x not in na_vals else np.nan)
    preg_df['birthwgt_oz'] = preg_df['birthwgt_oz'].apply(lambda x:x if x not in na_vals else np.nan)
    preg_df['totalwgt_lb'] = (preg_df['birthwgt_lb'] + preg_df[
        'birthwgt_oz']) / 16.0
    return preg_df

In [10]:
# cleaning the dataframe
preg_df = data_cleaning()
preg_df.head()

,caseid,prglength,outcome,pregordr,birthord,birthwgt_lb,birthwgt_oz,agepreg,finalwgt,totalwgt_lb
0,1,39.0,1,1,1.0,8.0,13.0,0.003316,6448.271112,1.3125
1,1,39.0,1,2,2.0,7.0,14.0,0.003925,6448.271112,1.3125
2,2,39.0,1,1,1.0,9.0,2.0,0.001433,12999.542264,0.6875
3,2,39.0,1,2,2.0,7.0,0.0,0.001783,12999.542264,0.4375
4,2,39.0,1,3,3.0,6.0,3.0,0.001833,12999.542264,0.5625


Count the number of times each value occurs.

In [11]:
preg_df.outcome.value_counts().sort_index()

1    9148
2    1862
3     120
4    1921
5     190
6     352
Name: outcome, dtype: int64

Check the values of another variable.

In [12]:
preg_df.birthwgt_lb.value_counts().sort_index()

0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
Name: birthwgt_lb, dtype: int64

Make a dictionary that maps from each respondent's `caseid` to a list of indices into the pregnancy `DataFrame`.  Use it to select the pregnancy outcomes for a single respondent.

In [12]:
caseid = 10229
preg_map = nsfg.MakePregMap(preg)
indices = preg_map[caseid]
preg.outcome[indices].values

array([4, 4, 4, 4, 4, 4, 1])

## Exercises

Select the `birthord` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611933)

In [13]:
# Solution goes here

We can also use `isnull` to count the number of nans.

In [14]:
preg.birthord.isnull().sum()

4445

Select the `prglngth` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611931)

In [15]:
# Solution goes here

To compute the mean of a column, you can invoke the `mean` method on a Series.  For example, here is the mean birthweight in pounds:

In [16]:
preg.totalwgt_lb.mean()

7.265628457623368

Create a new column named <tt>totalwgt_kg</tt> that contains birth weight in kilograms.  Compute its mean.  Remember that when you create a new column, you have to use dictionary syntax, not dot notation.

In [17]:
# Solution goes here

`nsfg.py` also provides `ReadFemResp`, which reads the female respondents file and returns a `DataFrame`:

In [18]:
resp = nsfg.ReadFemResp()

`DataFrame` provides a method `head` that displays the first five rows:

In [19]:
resp.head()

,caseid,rscrinf,rdormres,rostscrn,rscreenhisp,rscreenrace,age_a,age_r,cmbirth,agescrn,...,pubassis_i,basewgt,adj_mod_basewgt,finalwgt,secu_r,sest,cmintvw,cmlstyr,screentime,intvlngth
0,2298,1,5,5,1,5.0,27,27,902,27,...,0,3247.916977,5123.759559,5556.717241,2,18,1234,1222,18:26:36,110.492667
1,5012,1,5,1,5,5.0,42,42,718,42,...,0,2335.279149,2846.799490,4744.191350,2,18,1233,1221,16:30:59,64.294000
2,11586,1,5,1,5,5.0,43,43,708,43,...,0,2335.279149,2846.799490,4744.191350,2,18,1234,1222,18:19:09,75.149167
3,6794,5,5,4,1,5.0,15,15,1042,15,...,0,3783.152221,5071.464231,5923.977368,2,18,1234,1222,15:54:43,28.642833
4,616,1,5,4,1,5.0,20,20,991,20,...,0,5341.329968,6437.335772,7229.128072,2,18,1233,1221,14:19:44,69.502667


Select the `age_r` column from `resp` and print the value counts.  How old are the youngest and oldest respondents?

In [20]:
# Solution goes here

We can use the `caseid` to match up rows from `resp` and `preg`.  For example, we can select the row from `resp` for `caseid` 2298 like this:

In [21]:
resp[resp.caseid==2298]

,caseid,rscrinf,rdormres,rostscrn,rscreenhisp,rscreenrace,age_a,age_r,cmbirth,agescrn,...,pubassis_i,basewgt,adj_mod_basewgt,finalwgt,secu_r,sest,cmintvw,cmlstyr,screentime,intvlngth
0,2298,1,5,5,1,5.0,27,27,902,27,...,0,3247.916977,5123.759559,5556.717241,2,18,1234,1222,18:26:36,110.492667


And we can get the corresponding rows from `preg` like this:

In [22]:
preg[preg.caseid==2298]

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
2610,2298,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3247.916977,5123.759559,5556.717241,2,18,NaN,6.8750
2611,2298,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3247.916977,5123.759559,5556.717241,2,18,NaN,5.5000
2612,2298,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3247.916977,5123.759559,5556.717241,2,18,NaN,4.1875
2613,2298,4,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3247.916977,5123.759559,5556.717241,2,18,NaN,6.8750


How old is the respondent with `caseid` 1?

In [23]:
# Solution goes here

What are the pregnancy lengths for the respondent with `caseid` 2298?

In [24]:
# Solution goes here

What was the birthweight of the first baby born to the respondent with `caseid` 5012?

In [25]:
# Solution goes here